In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

In [2]:
from distutils.command.config import config


tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('./results/checkpoint-4500/pytorch_model.bin',config="./results/checkpoint-4500/config.json")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [3]:
data_path = "netflix_titles.csv"
df = pd.read_csv(data_path)
description = df['description']
listed_in =df["listed_in"]

In [4]:
import numpy as np
listed_in_arr = [ category.strip().lower().split(",") for category in listed_in]
dist_listed = []
temp =set()

listed_in_new = []
for category in listed_in_arr:
    temp_list = []
    for sub_cat in category:
        temp.add(sub_cat.strip())
        temp_list.append(sub_cat.strip())
    listed_in_new.append(temp_list)
dist_listed = list(temp)


print(len(dist_listed))
print(dist_listed)

42
['tv dramas', 'romantic tv shows', "kids' tv", 'comedies', 'sports movies', 'lgbtq movies', 'anime series', 'independent movies', 'anime features', 'horror movies', 'classic movies', 'stand-up comedy', 'music & musicals', 'tv horror', 'thrillers', 'korean tv shows', 'children & family movies', 'docuseries', 'crime tv shows', 'tv thrillers', 'classic & cult tv', 'romantic movies', 'spanish-language tv shows', 'international tv shows', 'tv mysteries', 'action & adventure', 'tv sci-fi & fantasy', 'movies', 'tv comedies', 'reality tv', 'cult movies', 'sci-fi & fantasy', 'teen tv shows', 'science & nature tv', 'documentaries', 'stand-up comedy & talk shows', 'faith & spirituality', 'tv action & adventure', 'british tv shows', 'tv shows', 'international movies', 'dramas']


In [5]:
print(len(listed_in_new))
cat_list_newArr  =[]
for cat_list in listed_in_new:
    temp_arr = []
    for cur_onehot in dist_listed:
        issame = False
        
        for sub_cat_list in cat_list:
            if cur_onehot==sub_cat_list:
                issame=True
                
        if issame:
            temp_arr.append(1)
        else:
            temp_arr.append(0)
    cat_list_newArr.append(np.array(temp_arr))

len(cat_list_newArr[2])

8807


42

In [6]:
num_of_flags = len(cat_list_newArr[0])
max_length = max([len(tokenizer.encode(title.strip())) for title in description])
print("max_length : ",max_length)

max_length :  62


In [7]:
len(description)

8807

### Sampel prediction

In [16]:
input_text = "Pat Sajak and Vanna White host one of TV's most popular"



input_flags = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]




for flag,cat in zip(input_flags,dist_listed):
    print(str(flag)+"\t:"+cat)

0	:tv dramas
0	:romantic tv shows
0	:kids' tv
0	:comedies
0	:sports movies
0	:lgbtq movies
0	:anime series
0	:independent movies
0	:anime features
0	:horror movies
0	:classic movies
0	:stand-up comedy
0	:music & musicals
0	:tv horror
0	:thrillers
0	:korean tv shows
0	:children & family movies
0	:docuseries
0	:crime tv shows
1	:tv thrillers
0	:classic & cult tv
0	:romantic movies
0	:spanish-language tv shows
0	:international tv shows
0	:tv mysteries
0	:action & adventure
1	:tv sci-fi & fantasy
0	:movies
0	:tv comedies
0	:reality tv
0	:cult movies
0	:sci-fi & fantasy
0	:teen tv shows
0	:science & nature tv
0	:documentaries
0	:stand-up comedy & talk shows
0	:faith & spirituality
0	:tv action & adventure
0	:british tv shows
0	:tv shows
0	:international movies
0	:dramas


In [17]:
i = 0
X = torch.tensor(input_flags)


encodings_dict = tokenizer('<|startoftext|>' + input_text + '<|endoftext|>',
                                       max_length=max_length, padding="max_length")
Y =  torch.tensor(encodings_dict['input_ids'])    

print("X ",tokenizer.decode(X, skip_special_tokens=True))
XY = torch.cat((Y,X), 0)
sample_input = torch.reshape(XY,(1,-1)).cuda()

sample_outputs = model.generate(sample_input,no_repeat_ngram_size = 1,num_beams=20, num_return_sequences=2,max_new_tokens=60)
print("Input : ",input_text)
print(len(sample_outputs[0]))

print("Generated output : ",tokenizer.decode(sample_outputs[0][:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


X  !!!!!!!!!!!!!!!!!!!"!!!!!!"!!!!!!!!!!!!!!!
Input :  Pat Sajak and Vanna White host one of TV's most popular
164
Generated output :  Pat Sajak and Vanna White host one of TV's most popular!!!!!!!!!!!!!!!!!!!"!!!!!!"!!!!!!!!!!!!!!!(As her father nears the end we all our hearts are in an attempt to help them both ways their lives will be answered by his co-worker Kirsten Johnson (@ki_strafingers.org) #WeAreWithYourOurVermot🇸😉❤️
